# Fine-Tuning Preparation
Based on the analysis, the confidence score is correlated with the number of labels predicted. This means that by increasing the number of predicted labels, the confidence score will increase as wel. However, this would only appply after the prediction.
The good news is that from the analysis, there are a few label types that have shown to have poor high confidence score ratio against low confidence score. This is while some low frequency labels have good ratio. So, to increase the confidence score with less label frequency. The training data needs to be augmented. Here, there are two things that can be done for this augmentation. These are:
- get contextual texts that corresponds to poor ratio labels.
- synthesize training data for rare texts.

------------
-----------
## Data Preparation

### Import Libraries

In [3]:
import pandas as pd
from gliner import GLiNER
from torch.utils.data import DataLoader
import torch
import os
import json
from sklearn.model_selection import train_test_split

C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Model

In [4]:
# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
print(f"Using device: {device}")

Using device: cpu


In [5]:

model = GLiNER.from_pretrained("urchade/gliner_multi")
model.model.to(device)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 57260.12it/s]
C:\Users\Arif Irfan Ibrahim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


SpanModel(
  (token_rep_layer): Encoder(
    (bert_layer): Transformer(
      (model): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(250105, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): Linear(in_features=768, out_features=768, bias=True)
                  (key_proj): Linear(in_features=768, out_features=768, bias=True)
                  (value_proj): Linear(in_features=768, out_features=768, bias=True)
                  (pos_dropout): Dropout(p=0.1, inplace=False)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): DebertaV2SelfOutput(
  

### Data Preparation

In [6]:
# config
data_path = "training_data.json"  # Path to your training data
output_dir = r"..\fine_tuned_model"   # Directory to save the fine-tuned model
batch_size = 4                    # Adjust based on your GPU memory
learning_rate = 2e-5              # Standard learning rate for fine-tuning
num_epochs = 10 

In [7]:
# load data
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [8]:
# data spliting (train/val/test)
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
val_ratio = 0.15
val_split = val_ratio / (1 - 0.2)  # 0.15 / 0.8 = 0.1875
train_data, val_data = train_test_split(train_val_data, test_size=val_split, random_state=42)

print(f"Total samples: {len(data)}")
print(f"Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")

Total samples: 2000
Train: 1300 | Val: 300 | Test: 400


### Training

In [9]:
model.fine_tune(
    train_data=train_data,
    val_data=val_data,
    output_dir=output_dir,
    batch_size=batch_size,
    learning_rate=learning_rate,
    num_epochs=num_epochs,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print(f"Model saved to {output_dir}")

AttributeError: 'GLiNER' object has no attribute 'fine_tune'

### Save Model

In [ ]:
model.save_pretrained(os.path.join(output_dir, "final_model"))
print("Training complete! Saved to:", output_dir)

### Evaluation

In [ ]:
if test_data:
    print("\nEvaluating on test set...")
    results = model.evaluate(test_data, batch_size=batch_size)
    print(f"Test F1: {results['f1']:.4f}, Precision: {results['precision']:.4f}, Recall: {results['recall']:.4f}")


Evaluating on test set...


KeyError: 'ner'